In [2]:
import os
import numpy as np

root_dir = "Dataset/snr_30"  # change to your root path
classes = sorted(os.listdir(root_dir))  # ['4ASK', '8ASK', 'OOK', ...]

X, y = [], []

for label_idx, cls in enumerate(classes):
    class_dir = os.path.join(root_dir, cls)
    for file in os.listdir(class_dir):
        if file.endswith(".npy"):
            filepath = os.path.join(class_dir, file)
            seq = np.load(filepath)  # shape: (1024, 2)
            X.append(seq)
            y.append(label_idx)

X = np.array(X)  # shape: (num_samples, 1024, 2)
y = np.array(y)
print(f"Loaded {len(X)} samples with shape {X.shape}")


Loaded 98304 samples with shape (98304, 1024, 2)


In [3]:
#Split Data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(X_train.shape, X_test.shape)


(78643, 1024, 2) (19661, 1024, 2)


In [4]:
#Create PyTorch Dataset and Dataloader
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Datasets
train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
#check shape
for xb, yb in train_loader:
    print(xb.shape, yb.shape)
    break

torch.Size([32, 1024, 2]) torch.Size([32])


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# reshape to 2D for fitting (combine all sequences)
X_reshaped = X.reshape(-1, X.shape[-1])  # (N*1024, 2)
X_scaled = scaler.fit_transform(X_reshaped)
X = X_scaled.reshape(X.shape)  # back to (N, 1024, 2)


In [ ]:
model = TimeSeriesTransformer(
    input_dim=2,          # I and Q
    model_dim=64,
    num_classes=len(classes),
    num_heads=4,
    num_layers=2
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop skeleton
for epoch in range(5):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}")
